# Structure your code with names

see images

1 упрощает чтение кода

2 граф становится более читаемым

3 не нужно писать длинные имена переменных

# Shared Variables

used for
1. input, output embedding
2. differeent model for train and test

# Simple network

In [1]:
# for using only cpu (not gpu)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

In [3]:
# load iris dataset (Ирисы Фишера)


TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

In [4]:
(train_x, train_y), (test_x, test_y) = load_data()

In [5]:
# create generator
def next_batch(X_data, Y_data, batch_size=10):
    length = X_data.shape[0]
    reduced_length = (length // batch_size) * batch_size
    
    X_data = X_data[:reduced_length].reshape([batch_size, -1, 4])
    Y_data = Y_data[:reduced_length].reshape([batch_size, -1]) 
    
    X_data = np.transpose(X_data, axes=(1, 0, 2))
    Y_data = np.transpose(Y_data, axes=(1, 0))
    for x_i, y_i in zip(X_data, Y_data):
        yield x_i, y_i

In [6]:
# create placeholders
batch_size = 10

X = tf.placeholder(tf.float32, shape=[batch_size, 4], name = 'input')
Y = tf.placeholder(tf.int32, shape=[batch_size], name = 'output')

In [7]:
# create simple network

layer_size = 50

# initialization - http://cs231n.github.io/neural-networks-2/#init
initializer = tf.random_normal_initializer(stddev=np.sqrt(2/layer_size), mean=0.0)

# create weight and bias
with tf.variable_scope(name_or_scope='layer_1'):
    w1 = tf.get_variable('weights', shape=[4,layer_size], initializer=initializer)
    b1 = tf.get_variable('bias', shape=[layer_size], initializer=initializer)
    output_from_layer = tf.nn.relu(tf.matmul(X, w1) + b1)

    
with tf.variable_scope(name_or_scope='layer_2'):
    w2 = tf.get_variable('weights', shape=[layer_size,3], initializer=initializer)
    b2 = tf.get_variable('bias', shape=[3], initializer=initializer)
    Y_predicted = tf.matmul(output_from_layer, w2) + b2

In [9]:
# use tf.reduce_mean for batches
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = Y_predicted, labels = Y), name='loss')


нарисуем граф

In [10]:
# choose optimizer to minimize loss
optimizer = tf.train.AdamOptimizer().minimize(loss)

# TAKE SUMMARY

In [11]:
# make summaries
tf.summary.scalar('loss', loss)
tf.summary.histogram('weights1', w1)
tf.summary.histogram('bias1', b1)

merged = tf.summary.merge_all()

In [12]:
# for name of summaries

from datetime import datetime
def time_str():
    return datetime.now().replace(microsecond=0).isoformat().replace(':','-')

в граф добавились узлы оптимизации

In [13]:
start = time.time()
with tf.Session() as sess:
    # initialize the necessary variables
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./iris/summuries%s' %time_str(), sess.graph)
    
    # train the model for 100 epochs
    for i in range(100):
        total_loss = 0
        for x_i, y_i in next_batch(train_x.values, train_y.values):
            summary, _, l = sess.run(
                [merged,
                 optimizer, 
                 loss],feed_dict= 
                {X:x_i, 
                 Y:y_i}) 
            
            total_loss += l  
        train_writer.add_summary(summary, global_step=i)
        if (i + 1) % 10 == 0:
            print('Epoch {0}: {1}'.format(i, total_loss))

Epoch 9: 7.0857910215854645
Epoch 19: 4.9306690990924835
Epoch 29: 3.757598102092743
Epoch 39: 2.9656500667333603
Epoch 49: 2.40960706025362
Epoch 59: 2.0175601691007614
Epoch 69: 1.7376424074172974
Epoch 79: 1.5337669402360916
Epoch 89: 1.3816078826785088
Epoch 99: 1.2654829286038876


see summaries in TensorBoard

for example: tensorboard --logdir="./iris/summuries2018-03-29T14-08-35/" --port 6006


In [17]:
# test our model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    print ('test' + '--'*20)
    print ('predicted -- true')

    for x_i, y_i in next_batch(test_x.values, test_y.values):
        Y_pred = sess.run(
            [Y_predicted],
            feed_dict= 
            {X:x_i})
        for index_i, Y_in_batch in enumerate(Y_pred[0]):
#             print (Y_in_batch)
            print ('{0} -- {1}'.format(np.argsort(Y_in_batch)[-1], y_i[index_i]))

test----------------------------------------
predicted -- true
1 -- 1
1 -- 1
0 -- 0
1 -- 2
1 -- 2
0 -- 0
0 -- 0
1 -- 1
0 -- 1
1 -- 1
1 -- 2
1 -- 1
1 -- 2
1 -- 2
1 -- 1
0 -- 1
1 -- 2
1 -- 2
1 -- 1
1 -- 2
0 -- 0
1 -- 1
1 -- 1
0 -- 0
0 -- 1
0 -- 0
0 -- 0
1 -- 1
0 -- 0
0 -- 1


In [20]:
# why so bad results?
# we test a network with random value in layers! because we initialized it again

In [22]:
start = time.time()
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        total_loss = 0
        for x_i, y_i in next_batch(train_x.values, train_y.values):
            _, l = sess.run(
                [optimizer, 
                 loss],
                feed_dict= 
                {X:x_i, 
                 Y:y_i}) 
            
            total_loss += l  
        if (i + 1) % 10 == 0:
            print('Epoch {0}: {1}'.format(i, total_loss))
            
    print ('test' + '--'*20)
    print ('predicted -- true')

    for x_i, y_i in next_batch(test_x.values, test_y.values):
        Y_pred = sess.run(
            [Y_predicted],
            feed_dict= 
            {X:x_i})
        for index_i, Y_in_batch in enumerate(Y_pred[0]):
#             print (Y_in_batch)
            print ('{0} -- {1}'.format(np.argsort(Y_in_batch)[-1], y_i[index_i]))

print('Took: %f seconds' %(time.time() - start))

Epoch 9: 7.24475422501564
Epoch 19: 5.384702891111374
Epoch 29: 4.360525518655777
Epoch 39: 3.5466431379318237
Epoch 49: 2.910990431904793
Epoch 59: 2.433864362537861
Epoch 69: 2.080517955124378
Epoch 79: 1.818356692790985
Epoch 89: 1.621521070599556
Epoch 99: 1.4709897227585316
test----------------------------------------
predicted -- true
1 -- 1
1 -- 1
0 -- 0
2 -- 2
2 -- 2
0 -- 0
0 -- 0
1 -- 1
1 -- 1
1 -- 1
2 -- 2
1 -- 1
1 -- 2
2 -- 2
1 -- 1
1 -- 1
2 -- 2
2 -- 2
1 -- 1
2 -- 2
0 -- 0
1 -- 1
1 -- 1
0 -- 0
1 -- 1
0 -- 0
0 -- 0
2 -- 1
0 -- 0
1 -- 1
Took: 1.133533 seconds


# Save  and restore model

tf.train.Saver.save(sess, save_path, global_step=None...)


tf.train.Saver.restore(sess, save_path)

In [23]:
saver = tf.train.Saver(max_to_keep=50)
# saver = tf.train.Saver({'variable': v1, 'v2': v2})


In [24]:
start = time.time()
with tf.Session() as sess:
    # Step 7: initialize the necessary variables, in this case, w and b
    sess.run(tf.global_variables_initializer())
    
    # Step 8: train the model for 100 epochs
    for i in range(100):
        total_loss = 0
        for x_i, y_i in next_batch(train_x.values, train_y.values, batch_size=10):
            _, l = sess.run(
                [optimizer, 
                 loss], 
                {X:x_i, 
                 Y:y_i}) 
            
            total_loss += l  
        if (i + 1) % 5 == 0:
            print('Epoch {0}: {1}'.format(i, total_loss))
            saver.save(sess, 
            './iris/model_network/model',
            global_step=i)


print('Took: %f seconds' %(time.time() - start))

Epoch 4: 9.786037623882294
Epoch 9: 7.0703694224357605
Epoch 14: 5.780054569244385
Epoch 19: 5.004405647516251
Epoch 24: 4.439634948968887
Epoch 29: 3.97695292532444
Epoch 34: 3.581219270825386
Epoch 39: 3.232933893799782
Epoch 44: 2.9260697960853577
Epoch 49: 2.6579074561595917
Epoch 54: 2.4251612573862076
Epoch 59: 2.224899247288704
Epoch 64: 2.053098790347576
Epoch 69: 1.9058754295110703
Epoch 74: 1.7795924842357635
Epoch 79: 1.670658115297556
Epoch 84: 1.5765843093395233
Epoch 89: 1.4948071464896202
Epoch 94: 1.4234918057918549
Epoch 99: 1.3609264269471169
Took: 1.923270 seconds


In [25]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, './iris/model_network/model-4')
    
    print ('test' + '--'*20)
    print ('predicted -- true')

    for x_i, y_i in next_batch(test_x.values, test_y.values):
        Y_pred = sess.run(
            [Y_predicted],
            feed_dict= 
            {X:x_i})
        for index_i, Y_in_batch in enumerate(Y_pred[0]):
#             print (Y_in_batch)
            print ('{0} -- {1}'.format(np.argsort(Y_in_batch)[-1], y_i[index_i]))

INFO:tensorflow:Restoring parameters from ./iris/model_network/model-4
test----------------------------------------
predicted -- true
2 -- 1
2 -- 1
0 -- 0
2 -- 2
2 -- 2
0 -- 0
0 -- 0
2 -- 1
2 -- 1
2 -- 1
2 -- 2
2 -- 1
2 -- 2
2 -- 2
2 -- 1
2 -- 1
2 -- 2
2 -- 2
2 -- 1
2 -- 2
0 -- 0
2 -- 1
2 -- 1
0 -- 0
2 -- 1
0 -- 0
0 -- 0
2 -- 1
0 -- 0
2 -- 1
